In [1]:
from shower import shower2x2
import sys
sys.path.append('../')

from src.pyprop import multimodel_rep, OI_calc

import numpy as np

In [2]:
DOS_bounds =  np.array([[10, 20], 
                        [70, 100]])

AIS_bounds =  np.array([[0.1, 10],
                        [0.1, 10]])

AIS_resolution =  [20, 20]

In [3]:
from src.pyprop import AIS2AOS_map

In [4]:
AIS, AOS =  AIS2AOS_map(shower2x2, AIS_bounds, AIS_resolution)

In [5]:
AIS =  AIS.reshape(-1,2)
AOS =  AOS.reshape(-1,2)

In [6]:
import matplotlib.pyplot as plt

In [7]:
plt.scatter(AIS[:,0], AIS[:,1])

In [8]:
plt.scatter(AOS[:,0], AOS[:,1])

In [16]:
rng = np.random.RandomState(0)
training_indices = rng.choice(np.arange(AOS.shape[0]), size=50, replace=False)
training_indices = np.sort(training_indices)
X_train, y_train = AIS[training_indices], AOS[training_indices, 0]
inverted_idx = [x not in training_indices for x in range(0, len(AIS))]
X_val, y_val = AIS[inverted_idx], AOS[inverted_idx, 0]



In [17]:
print(training_indices)

[  1   6  12  15  37  54  59  60  65  76  78  90 100 102 113 124 132 134
 140 141 155 158 170 173 175 176 179 196 199 206 214 225 233 246 255 261
 268 286 293 297 303 309 313 322 334 341 344 348 374 399]


In [18]:
print(inverted_idx)

[True, False, True, True, True, True, False, True, True, True, True, True, False, True, True, False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, False, True, True, True, True, False, False, True, True, True, True, False, True, True, True, True, True, True, True, True, True, True, False, True, False, True, True, True, True, True, True, True, True, True, True, True, False, True, True, True, True, True, True, True, True, True, False, True, False, True, True, True, True, True, True, True, True, True, True, False, True, True, True, True, True, True, True, True, True, True, False, True, True, True, True, True, True, True, False, True, False, True, True, True, True, True, False, False, True, True, True, True, True, True, True, True, True, True, True, True, True, False, True, True, False, True, True, True, True,

In [20]:
print(y_train)

[ 0.72105  3.32632  6.45263  8.01579  9.57895  8.53684 11.14211  1.76316
  4.36842 10.1     11.14211  7.49474  2.80526  3.84737  9.57895  5.41053
  9.57895 10.62105  3.84737  4.36842 11.66316 13.22632  9.57895 11.14211
 12.18421 12.70526 14.26842 13.22632 14.78947  8.53684 12.70526  8.53684
 12.70526  9.57895 14.26842  7.49474 11.14211 10.62105 14.26842 16.35263
  9.57895 12.70526 14.78947  9.57895 15.83158  9.57895 11.14211 13.22632
 16.87368 20.     ]


In [21]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF

In [22]:
kernel = RBF()

In [23]:
gaussian_process = GaussianProcessRegressor(kernel=kernel)

In [24]:
gaussian_process.fit(X_train, y_train)

GaussianProcessRegressor(kernel=RBF(length_scale=1))

In [25]:
mean_prediction, std_prediction = gaussian_process.predict(X_val, return_std=True)

In [26]:
print(gaussian_process.kernel_)

RBF(length_scale=3.92)


In [36]:
ax = plt.figure().add_subplot(projection='3d')

In [37]:
ax.scatter3D(X_val[:,0], X_val[:,1], y_val, label='shower problem y1', linestyle=
        'dotted')

In [38]:
ax2 = plt.figure().add_subplot(projection='3d')

ax2.scatter3D(X_val[:,0], X_val[:,1], mean_prediction, label='shower problem y1 - GP', linestyle=
        'dotted')

In [30]:
print(mean_prediction)

[ 0.21541  1.23803  1.75981  2.28264  2.80492  3.84713  4.36777  4.88857
  5.40967  5.93107  6.97412  7.49525  8.53574  9.05535  9.57489 10.09393
  0.72491  1.23601  1.75603  2.27926  2.80272  3.32534  3.84709  4.36833
  4.88936  5.41035  5.93136  6.45241  6.97353  7.4947   8.01586  8.53694
  9.05793 10.1     10.6204   1.24251  1.75721  2.27866  2.80201  3.325
  3.84708  4.36853  4.88972  5.41089  5.93203  6.45308  6.97401  7.49488
  8.01581  9.05798  9.5792  10.10052 10.62178  2.28026  2.80226  3.3251
  3.84718  4.88932  5.41033  5.93157  6.4529   6.97413  7.49515  8.01602
  8.53688  9.05783  9.57888 10.62115 11.66188  2.28433  2.80309  3.32525
  3.84744  4.36864  4.88913  5.40957  5.93042  6.45174  6.97328  8.01596
  8.53697  9.0579   9.57884 10.09981 10.62078 11.1417  11.66235 12.18169
  3.3252   4.36904  4.88962  5.40963  5.92983  6.45064  6.97205  7.49375
  8.01538  8.53676  9.05792 10.09993 10.62087 11.14176 11.66258 12.18316
 12.70248  3.32614  3.84684  4.36894  4.89026  5.93039

In [31]:
from sklearn.metrics import r2_score
r2 =  r2_score(y_val, mean_prediction)
print(r2)

0.9999985773314278


In [39]:
np.random.randn(100)

array([ 1.55364, -0.48592,  0.06185,  0.35627, -0.51854,  0.43338,
       -1.27945, -0.73825, -0.12546,  0.17926,  0.62969, -1.27254,
       -0.04803, -0.07438,  0.90433,  0.68275,  0.7526 ,  0.0204 ,
        2.70751,  0.32421, -0.13113,  0.27373,  0.99953,  0.5894 ,
        0.26535,  0.61254,  1.56436, -1.13769, -0.83465, -1.39597,
        0.89254,  1.43032, -0.08737,  0.97981, -1.13819, -0.09767,
       -0.98361, -0.10745, -0.58197, -1.62752, -0.69359, -0.263  ,
       -1.18389,  0.9916 ,  1.85399,  0.3438 ,  0.81195, -0.17072,
        0.48225,  0.17744,  1.10144,  0.52598, -0.29347, -0.66484,
       -0.25232,  0.08095,  0.46094, -0.66818, -0.42389,  0.64239,
       -1.22876, -1.55856,  1.38327, -0.0716 , -0.38647, -0.05263,
        0.79692, -0.02626,  2.24266,  0.34424, -0.68893,  0.01355,
        1.49705,  0.82662,  0.47537,  1.52173, -0.62022, -0.51687,
        0.58437,  0.38642,  1.55898, -0.1764 ,  0.08115,  0.71686,
       -2.1167 , -0.42695,  0.03491, -0.46781, -0.7119 , -0.88

In [42]:
y_train2 =  y_train + np.random.randn(y_train.shape[0])

In [43]:
print(y_train2)

[ 0.67463  3.60694  6.3689   8.02279  9.95164  9.4864  12.47557  1.34417
  3.64963  8.26807 10.54264  8.60149  2.72834  3.73073  7.60093  6.00556
  9.21412 10.47078  5.1426   5.87749 11.2079  13.90458 10.93582 12.17562
 12.11897 12.19772 14.21873 14.11994 13.60872  8.86843 13.86968  7.87796
 12.76513  9.87614 13.71862  8.30842 12.18641 10.79282 14.94749 15.33963
  8.67908 11.78932 14.29264 10.44063 14.31012  9.04563 11.25312 14.23531
 18.28612 19.14751]
